In [ ]:
import glob
import pandas as pd
from datetime import datetime
import xml.etree.ElementTree as ET
import wget
import shutil
import requests

## Download Files

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip"
response = requests.get(url)

if response.status_code == 200:
    with open("source.zip", "wb") as f:
        f.write(response.content)
        print("file downloaded")
else:
    print("Failed to download the file.")

## Unzip Files

In [ ]:
shutil.unpack_archive('source.zip', './')

## See unzipped files

In [ ]:
files = glob.glob("C:/Users/X1/jupytor_python/Data Engineering/*")
for file in files:
    if file.endswith(('.json', '.csv','xml')):
        print(file)

In [ ]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

### CSV Extract Function

In [ ]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### json Extract Function

In [ ]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### xml Extract Function

In [ ]:
def extract_from_xml(file_to_process):
    data = {"name": [], "height": [], "weight": []}
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        data["name"].append(name)
        data["height"].append(height)
        data["weight"].append(weight)
    return pd.DataFrame(data)

### Extract Function

In [ ]:
def extract():
    extracted_data = pd.DataFrame(columns=['name', 'height', 'weight'])

    # Process all CSV files
    for csvfile in glob.glob("*.csv"):
        extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True)

    # Process all JSON files
    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True)

    # Process all XML files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True)

    return extracted_data

## Transform

The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms

In [ ]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading

In [ ]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile) 

## Logging

In [ ]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process

In [ ]:
log("ETL Job Started")

In [ ]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

In [ ]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

In [ ]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [ ]:
log("ETL Job Ended")